In [31]:
import arcpy

# set initial workspace
arcpy.env.workspace = r'C:\Users\bryantch\Documents\ArcGIS\Projects\hexbin_parcel_analysis\hexbin_parcel_analysis.gdb'
# create lists of the annual variable fields

attribute_names = ['resunits', 'homestead', 'nonressf', 'pyr_market', 'pyr_taxes']

In [34]:
# For each variable of interest, transpose the table to pull out one row for each year for each hexagram. This will produce a separate table for each variable.

for name in attribute_names:
    in_table_transp = fr'C:\Users\bryantch\Documents\ArcGIS\Projects\hexbin_parcel_analysis\hexbin_parcel_analysis.gdb\attribute_FCs\{name}_hexbins'
    in_field_transp = f'{name}_2009_n 2009; {name}_2010_n 2010; {name}_2011_n 2011; {name}_2012_n 2012; {name}_2013_n 2013; {name}_2014_n 2014; \
        {name}_2015_n 2015; {name}_2016_n 2016; {name}_2017_n 2017; {name}_2018_n 2018; {name}_2019_n 2019; {name}_2020_n 2020; {name}_2021_n 2021; \
        {name}_2022_n 2022'
    out_table_transp = f'{name}_transp' 
    in_transposed_field_name = 'year' 
    in_value_field_name = f'{name}'
    attribute_fields = ['SHAPE', 'OBJECTID']
    
    # delete out_table if it already exists
    if arcpy.Exists(out_table_transp):
        arcpy.Delete_management(in_table_transp)


    arcpy.management.TransposeFields(in_table_transp, in_field_transp, out_table_transp, in_transposed_field_name, in_value_field_name, attribute_fields)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Table: Dataset C:\Users\bryantch\Documents\ArcGIS\Projects\hexbin_parcel_analysis\hexbin_parcel_analysis.gdb\attribute_FCs\resunits_hexbins does not exist or is not supported
ERROR 000725: Output Table: Dataset C:\Users\bryantch\Documents\ArcGIS\Projects\hexbin_parcel_analysis\hexbin_parcel_analysis.gdb\resunits_transp already exists.
Failed to execute (TransposeFields).


In [ ]:
# For each new table, calculate a new field by copying 'year' into a new date type field. 

for name in attribute_names:
    in_table_date = f'{name}_transp'
    input_time_field = 'year'
    input_time_format = 'yyyy'
    output_time_field = 'yearDate'
    output_time_type = 'DATE'

    arcpy.management.ConvertTimeField(in_table_date, input_time_field, input_time_format, output_time_field, output_time_type)

In [ ]:
# Space Time Cube requires an integer field for location

for name in attribute_names:
    inFeatures_oid = f'{name}_transp'
    new_field_name_oid = 'OBJECTID_int'
    expression_oid = '!OBJECTID!'

    arcpy.management.CalculateField(inFeatures_oid, new_field_name_oid, expression_oid, "PYTHON3", field_type='LONG')

In [ ]:
# Space Time Cube requires a number variable to go into each bin, so we need to convert the attribute field into a number type

for name in attribute_names:
    inFeatures_num = f'{name}_transp'
    new_field_name_num = f'{name}_FLT'
    expression_num = f'!{name}!'

    arcpy.management.CalculateField(inFeatures_num, new_field_name_num, expression_num, "PYTHON3", field_type='FLOAT')

In [30]:
try:

    # create space time cube for each attribute
    for name in attribute_names:

        in_features_cube = f'{name}_transp'
        output_cube = rf'C:\Users\bryantch\Documents\ArcGIS\Projects\hexbin_parcel_analysis\{name}_STCube.nc'
        location_id = 'OBJECTID_int'
        temporal_aggregation = 'NO_TEMPORAL_AGGREGATION'
        time_field = 'yearDate'
        variable_name = f'{name}_FLT ZEROS'  

        arcpy.stpm.CreateSpaceTimeCubeDefinedLocations(in_features_cube, output_cube, location_id, temporal_aggregation, time_field, time_step_interval= '1 Years', \
                                                    time_step_alignment='END_TIME', variables=variable_name)
except arcpy.ExecuteError:
    print(arcpy.GetMessages())

Start Time: Monday, October 9, 2023 3:04:38 PM
Failed to execute. Parameters are not valid.
ERROR 000308: Invalid field type
Failed to execute (CreateSpaceTimeCubeDefinedLocations).
Failed at Monday, October 9, 2023 3:04:38 PM (Elapsed Time: 0.01 seconds)


In [ ]:
# for each Space Time Cube, do an emerging hotspot/coldspot analysis

for name in attribute_names:

    in_cube = fr'C:\Users\bryantch\Documents\ArcGIS\Projects\hexbin_parcel_analysis\hexbin_parcel_analysis.gdb\{name}_STCube.nc'
    name = name.upper # 'create spacetime cube' tool appears to force variable name into all caps, so need to adjust to avoid error
    analysis_variable = f'{name}_FLT_NONE_SPACE_TIME_NEIGHBORS' #getting error on this because it needs to be all caps. Ran all manually but needs to be fixed.
    output_features = f'{name}_STCube_EmergingHotSpotAnalysis'

    arcpy.stpm.EmergingHotSpotAnalysis(in_cube, analysis_variable, output_features, conceptualization_of_spatial_relationships='FIXED_DISTANCE', define_global_window='ENTIRE_CUBE')